In [1]:
# !pip install ultralytics pandas pyarrow -q

## Load Model

In [2]:
import cv2
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

## Load Dataset

In [3]:
import json
import pandas as pd

folder_path = '/workspace/datasets/coco_minitrain_25k/annotations/'
file_name = 'instances_minitrain2017.json'

file_path = folder_path + file_name
    
with open(file_path, 'r') as file:
    data = json.load(file)


df = pd.DataFrame(data['images'])
df = df[['file_name', 'id']]
df

,file_name,id
0,000000231538.jpg,231538
1,000000322749.jpg,322749
2,000000318299.jpg,318299
3,000000025290.jpg,25290
4,000000020276.jpg,20276
...,...,...
24995,000000465301.jpg,465301
24996,000000281503.jpg,281503
24997,000000260893.jpg,260893
24998,000000405192.jpg,405192


## Run Inference

In [4]:
import cv2
from ultralytics import YOLO
from tqdm.auto import tqdm

det_boxes = []
det_confidences = []
det_class_ids = []
filename = []
id = []

# df = df.head(10)

img_path = '/workspace/datasets/coco_minitrain_25k/images/train2017/'
for row in tqdm(df.itertuples(), desc="Processing", total=len(df), unit=" imgs"):
    image_path = img_path + row.file_name
    image = cv2.imread(image_path)
    results = model(image, verbose=False)[0]

    filename.append(row.file_name)
    id.append(row.id) 
    det_boxes.append(results.boxes.xywh.cpu().numpy())
    det_confidences.append(results.boxes.conf.cpu().numpy())
    det_class_ids.append(results.boxes.cls.cpu().numpy())

yolo_detections_df = pd.DataFrame({'filename': filename, 
                                   'id':id,
                                   'det_boxes': det_boxes, 
                                   'det_conf': det_confidences, 
                                   'det_category_id': det_class_ids})

yolo_detections_df['det_boxes'] = yolo_detections_df['det_boxes'].apply(lambda x: [[round(float(y), 2) for y in sublist] for sublist in x])
yolo_detections_df['det_conf'] = yolo_detections_df['det_conf'].apply(lambda x: [round(num, 4) for num in x])
yolo_detections_df['det_category_id'] = yolo_detections_df['det_category_id'].apply(lambda x: [int(num) for num in x])

yolo_detections_df

Processing:   0%|          | 0/25000 [00:00<?, ? imgs/s]

,filename,id,det_boxes,det_conf,det_category_id
0,000000231538.jpg,231538,"[[369.12, 374.82, 94.64, 367.55], [275.72, 360...","[0.8767, 0.8523, 0.7863, 0.7315, 0.6548, 0.618...","[0, 0, 0, 28, 0, 24, 28, 0, 28]"
1,000000322749.jpg,322749,"[[194.15, 80.24, 183.24, 151.9], [582.14, 249....","[0.7612, 0.7044, 0.5148, 0.4506, 0.4182, 0.326...","[50, 50, 51, 50, 50, 51, 51]"
2,000000318299.jpg,318299,"[[458.6, 221.3, 362.52, 239.65], [178.15, 203....","[0.9268, 0.8947, 0.2532]","[7, 7, 7]"
3,000000025290.jpg,25290,"[[460.87, 165.48, 334.83, 326.56], [540.66, 33...","[0.9171, 0.8854]","[0, 36]"
4,000000020276.jpg,20276,"[[292.33, 211.56, 211.44, 191.84], [177.89, 30...","[0.8757, 0.8557]","[0, 37]"
...,...,...,...,...,...
24995,000000465301.jpg,465301,"[[283.26, 280.91, 88.91, 123.57], [69.65, 300....","[0.8988, 0.8958, 0.7998, 0.5443, 0.5305, 0.394...","[56, 57, 56, 56, 69, 56, 60, 72, 68, 56]"
24996,000000281503.jpg,281503,"[[345.71, 114.32, 68.75, 142.11], [240.0, 237....","[0.9087, 0.8906, 0.88, 0.8417, 0.7915, 0.6311,...","[0, 0, 32, 0, 0, 0, 34, 0, 34, 0]"
24997,000000260893.jpg,260893,"[[157.85, 202.96, 161.85, 312.32], [182.92, 19...","[0.9464, 0.464]","[76, 76]"
24998,000000405192.jpg,405192,"[[179.49, 475.72, 355.93, 328.03], [387.29, 44...","[0.9233, 0.8982, 0.8885, 0.8275, 0.7691, 0.658...","[53, 42, 0, 60, 0, 40, 53, 56, 41]"


## Save Results

In [5]:
yolo_detections_df.to_parquet('yolov8n_detection_results.parquet')